In [1]:
import pandas as pd


In [2]:
dataset = pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset = pd.get_dummies(dataset, dtype = int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
#assign input and output columns
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [8]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [9]:
dependent=dataset[['charges']]

In [10]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [11]:

#splitting Train set and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
 #from sklearn.tree import DecisionTreeRegressor
param_grid = {'criterion':['friedman_mse','poisson'],
'max_features': ['sqrt','log2'],
 'n_estimators':[10,100]}
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose = 3,n_jobs=-1)
 # fitting the model for grid search
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\Dharani Vinoth\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse', 'poisson'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             verbose=3)

In [25]:
# print best parameter after tuning
 #print(grid.best_params_)
re=grid.cv_results_
 #print(re)
grid_predictions = grid.predict(X_test)
 # print classification report
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.
best_params_),r_score)

The R_score value for best parameter {'criterion': 'poisson', 'max_features': 'log2', 'n_estimators': 100}: 0.8676439067681122


In [26]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.043580,0.001600,0.003001,2.861023e-07,friedman_mse,sqrt,10,"{'criterion': 'friedman_mse', 'max_features': ...",0.825852,0.782167,0.795363,0.795705,0.748610,0.789540,0.024980,6
1,0.383777,0.003524,0.012520,9.822558e-04,friedman_mse,sqrt,100,"{'criterion': 'friedman_mse', 'max_features': ...",0.863555,0.797149,0.805705,0.825285,0.773478,0.813034,0.030243,2
2,0.041774,0.001025,0.003296,1.255331e-03,friedman_mse,log2,10,"{'criterion': 'friedman_mse', 'max_features': ...",0.820323,0.762049,0.791246,0.775851,0.738106,0.777515,0.027626,8
3,0.383499,0.004210,0.012516,7.369600e-04,friedman_mse,log2,100,"{'criterion': 'friedman_mse', 'max_features': ...",0.859582,0.794386,0.806561,0.828390,0.770153,0.811814,0.030409,3
4,0.044880,0.001624,0.003506,2.749652e-04,poisson,sqrt,10,"{'criterion': 'poisson', 'max_features': 'sqrt...",0.852068,0.780324,0.800866,0.812789,0.761991,0.801608,0.030641,5
5,0.407092,0.007743,0.011304,9.713799e-03,poisson,sqrt,100,"{'criterion': 'poisson', 'max_features': 'sqrt...",0.861616,0.793427,0.807740,0.824306,0.771889,0.811796,0.030282,4
6,0.045525,0.002014,0.003363,7.655345e-04,poisson,log2,10,"{'criterion': 'poisson', 'max_features': 'log2...",0.820143,0.784407,0.791388,0.803159,0.745932,0.789006,0.024706,7
7,0.387711,0.008044,0.006391,7.827358e-03,poisson,log2,100,"{'criterion': 'poisson', 'max_features': 'log2...",0.862248,0.791500,0.815737,0.833397,0.769751,0.814527,0.032156,1


In [29]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
#change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Age:56
BMI:70
Children:2
Sex Male 0 or 1:1
Smoker Yes 0 or 1:0
Future_Prediction=[14871.5277141]
